### 1. INGEST & CLEAN

In [0]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp

# Create a SparkSession with explicit configuration
spark = SparkSession.builder \
    .appName("Urban Green Space Management System") \
    .getOrCreate()

# URLs for the datasets
aq_url = "https://raw.githubusercontent.com/Dilshan-Chanuka/ml-anomaly-detection-pipeline/refs/heads/main/data_sample/national_parks_air_quality.csv"
foot_url = "https://raw.githubusercontent.com/Dilshan-Chanuka/ml-anomaly-detection-pipeline/refs/heads/main/data_sample/national_parks_footfall.csv"
sent_url = "https://raw.githubusercontent.com/Dilshan-Chanuka/ml-anomaly-detection-pipeline/refs/heads/main/data_sample/national_parks_sentiment.csv"
parks_url = "https://raw.githubusercontent.com/Dilshan-Chanuka/ml-anomaly-detection-pipeline/refs/heads/main/data_sample/german_national_parks.csv"

def read_data(url):
    """Read CSV data from a URL into a pandas DataFrame and then convert it to a Spark DataFrame"""
    return spark.createDataFrame(pd.read_csv(url))
    
def clean_data(df):
    """Clean and format the DataFrame"""
    df = df.dropna(how='any', subset=[col for col in df.columns if col != ''])
    if 'timestamp' in df.columns:
        df = df.withColumn("timestamp", to_timestamp(col("timestamp"), "M/d/yyyy H:mm"))
    if 'visitor_count' in df.columns:
        df = df.withColumn("visitor_count", col("visitor_count").cast("integer"))
    if 'event_day' in df.columns:
        df = df.withColumn("event_day", col("event_day").cast("boolean"))
    return df

def write_to_delta(df, table_name):
    """Write the DataFrame to a Delta table"""
    try:
        df.write.format("delta") \
            .mode("overwrite") \
            .option("overwriteSchema", "true") \
            .saveAsTable(table_name)
    except Exception as e:
        print(f"Error writing to Delta table {table_name}: {e}")

# Read and clean data
aq_df = clean_data(read_data(aq_url))
foot_df = clean_data(read_data(foot_url))
sent_df = clean_data(read_data(sent_url))
parks_df = clean_data(read_data(parks_url))

# Optionally display the DataFrames
display(aq_df)
display(foot_df)
display(sent_df)
display(parks_df)

# Write to Delta tables
write_to_delta(aq_df, "ml_project.bronze.air_quality")
write_to_delta(foot_df, "ml_project.bronze.footfall")
write_to_delta(sent_df, "ml_project.bronze.sentiment")
write_to_delta(parks_df, "ml_project.bronze.parks")

In [0]:
# Perform EDA on the DataFrames
print("Air Quality DataFrame Summary:")
aq_df.describe().show()

print("Footfall DataFrame Summary:")
foot_df.describe().show()

print("Sentiment DataFrame Summary:")
sent_df.describe().show()

print("Parks DataFrame Summary:")
parks_df.describe().show()

# Visualize the data using plots
import matplotlib.pyplot as plt

# Plot the distribution of visitor_count
foot_df_pd = foot_df.toPandas()
plt.hist(foot_df_pd['visitor_count'], bins=50)
plt.title('Distribution of Visitor Count')
plt.xlabel('Visitor Count')
plt.ylabel('Frequency')
plt.show()

# Plot the distribution of sentiment_score
sent_df_pd = sent_df.toPandas()
plt.hist(sent_df_pd['sentiment_score'], bins=50)
plt.title('Distribution of Sentiment Score')
plt.xlabel('Sentiment Score')
plt.ylabel('Frequency')
plt.show()

In [0]:
# Create new features
from pyspark.sql.functions import dayofweek, hour

# Extract day of week and hour from timestamp
foot_df = foot_df.withColumn("day_of_week", dayofweek(col("timestamp"))) \
                  .withColumn("hour", hour(col("timestamp")))

# Create a new feature: average visitor count per day of week
avg_visitor_count_per_day = foot_df.groupBy("day_of_week").agg({"visitor_count": "mean"}).withColumnRenamed("avg(visitor_count)", "avg_visitor_count")
foot_df = foot_df.join(avg_visitor_count_per_day, on="day_of_week", how="left")

# Create a new feature: sentiment score per park
sentiment_score_per_park = sent_df.groupBy("park_id").agg({"sentiment_score": "mean"}).withColumnRenamed("avg(sentiment_score)", "avg_sentiment_score")
parks_df = parks_df.join(sentiment_score_per_park, on="park_id", how="left")

display(foot_df)
display(parks_df)

In [0]:
from pyspark.sql.functions import to_date, count, when, col, dayofweek, hour, mean
import pyspark.sql.functions as F

# Convert the date column to date type
aq_df = aq_df.withColumn("date", to_date(col("date"), "M/d/yyyy"))

# Convert the timestamp column to date type
foot_df = foot_df.withColumn("date", to_date(col("timestamp")))

# Create new features
foot_df = foot_df.withColumn("day_of_week", dayofweek(col("timestamp"))) \
                  .withColumn("hour", hour(col("timestamp")))

# Create a new feature: average visitor count per day of week
avg_visitor_count_per_day = foot_df.groupBy("day_of_week").agg(F.mean("visitor_count").alias("avg_visitor_count"))
foot_df = foot_df.join(avg_visitor_count_per_day, on="day_of_week", how="left")

# Create a new feature: sentiment score per park
sentiment_score_per_park = sent_df.groupBy("park_id").agg(F.mean("sentiment_score").alias("avg_sentiment_score"))
parks_df = parks_df.join(sentiment_score_per_park, on="park_id", how="left")

# Integrate the DataFrames
integrated_df = foot_df.join(parks_df, on="park_id", how="left") \
                        .join(aq_df, on=["park_id", "date"], how="left") \
                        .join(sent_df, on=["park_id", "timestamp"], how="left")

# Rename columns to avoid invalid characters
for col_name in integrated_df.columns:
    if '(' in col_name or ')' in col_name:
        new_col_name = col_name.replace('(', '_').replace(')', '_')
        integrated_df = integrated_df.withColumnRenamed(col_name, new_col_name)

# Drop duplicate columns
for col_name in integrated_df.columns:
    if integrated_df.columns.count(col_name) > 1:
        integrated_df = integrated_df.drop(col_name)

# Save the integrated DataFrame as a Delta table
write_to_delta(integrated_df, "ml_project.silver.integrated")
display(integrated_df)
